In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
import glob
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/vaishalee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vaishalee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vaishalee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
input_files = pd.DataFrame(pd.read_excel('Input.xlsx'))

In [7]:
input_files.head()

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...


In [8]:
output_file=pd.DataFrame()
output_file['URL_ID']=input_files['URL_ID']
output_file['URL']=input_files['URL']

In [9]:
##### Code for content scraping from URLs
def content_scraping(x):
  url = x
  reqs = requests.get(url)
  soup = BeautifulSoup(reqs.text, 'html.parser')
  title=soup.find('title').get_text()

  content=soup.find("div", {"class": "td-post-content tagdiv-type"})
  if content is not None:
    content=content.get_text()

  if content is None:
    content=soup.find_all("div", class_="tdb-block-inner td-fix-index")
    max_val=-1
    flag=-1
    numb=-1

    for i in content:
      numb=numb+1
      temp_con=i.get_text()
      if max_val<len(temp_con):
        max_val=len(temp_con)
        flag=numb

    content=content[flag].get_text()

  content=title+"\n"+content
  return content

In [10]:
all_content=[]
for ind in input_files['URL'].index:
  print(ind)
  curr_content=content_scraping(input_files['URL'][ind])
  all_content.append(curr_content)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [11]:
output_file['Data']=all_content

In [12]:
##### Code to Create txt files of the web scraped content with filename as the URL ID
# for ind in output_file.index:
#   filename=base_location+output_file['URL_ID'][ind]+'.txt'
#   f = open(filename, "w")
#   f.write(output_file['Data'][ind])

In [13]:
##### Loading the stopwords given in the project folder
file_list = glob.glob('StopWords/*.txt')

stopwords = []

for file_path in file_list:
    
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        
        stopwords.extend(f.read().lower().splitlines())

In [14]:
pos_words= []
with open('MasterDictionary/positive-words.txt', 'r', encoding='utf-8', errors='ignore') as f:
  pos_words.extend(f.read().lower().splitlines())

In [15]:
neg_words= []
with open('MasterDictionary/negative-words.txt', 'r', encoding='utf-8', errors='ignore') as f:
  neg_words.extend(f.read().lower().splitlines())

In [19]:
##### Preprocessing the text by changing it to lower case, removing all punctuation,
##### tokenizing it and removing the stopwords and saving it to a column in
##### dataframe for further use
def text_prep(x):
    corp = str(x).lower()
    corp = re.sub('[^a-zA-Z]+', ' ', corp).strip()
    tokens = word_tokenize(corp)
    words = [t for t in tokens if t not in stopwords]
    return words

In [20]:
output_file['Processed']= output_file['Data'].apply(text_prep)

In [21]:
##### Defining functions to calculate positive, nagative and polarity score
def positive(words):
    pos_count=len([i for i in words if i in pos_words])
    return pos_count

In [22]:
def negative(words):
    neg_count=len([i for i in words if i in neg_words])
    return neg_count

In [23]:
def polarity(positive, negative):
  pol_score=(positive-negative)/(positive+negative+0.000001)
  return pol_score

In [24]:
##### Using the defined functions to calculate the positive, negative and polarity
##### score and storing it in columns in the dataframe
output_file['POSITIVE SCORE']= output_file['Processed'].apply(positive)

In [25]:
output_file['NEGATIVE SCORE']= output_file['Processed'].apply(negative)

In [26]:
output_file['POLARITY SCORE']= output_file.apply(lambda row: polarity(row['POSITIVE SCORE'], row['NEGATIVE SCORE']), axis=1)

In [27]:
##### Defining function to calculate subjectivity score and saving it in dataframe
def subjectivity(positive, negative, cleaned_words_number):
  sub_score=(positive+negative)/(cleaned_words_number+0.000001)
  return sub_score

In [28]:
output_file['SUBJECTIVITY SCORE']= output_file.apply(lambda row: subjectivity(row['POSITIVE SCORE'], row['NEGATIVE SCORE'], len(row['Processed'])), axis=1)

In [29]:
##### Defining function to calculate average sentence length and saving it in
##### dataframe
def avg_sentence_len(content, words):
  sentences = sent_tokenize(content)
  total_sent = len(sentences)
  total_words = len(words)
  avg_sentence_length = total_words / total_sent if total_sent > 0 else 0
  return avg_sentence_length

In [30]:
output_file['AVG SENTENCE LENGTH']= output_file.apply(lambda row: avg_sentence_len(row['Data'], row['Processed']), axis=1)

In [31]:
##### Function to calculate the number of syllables in a given word according to
##### accepted rules of the english language
def sylco(word) :
    word = word.lower()

    # exception_add are words that need extra syllables
    # exception_del are words that need less syllables

    exception_add = ['serious','crucial']
    exception_del = ['fortunately','unfortunately']

    co_one = ['cool','coach','coat','coal','count','coin','coarse','coup','coif','cook','coign','coiffe','coof','court']
    co_two = ['coapt','coed','coinci']

    pre_one = ['preach']

    syls = 0 #added syllable number
    disc = 0 #discarded syllable number

    #1) if letters < 3 : return 1
    if len(word) <= 3 :
        syls = 1
        return syls

    #2) if doesn't end with "ted" or "tes" or "ses" or "ied" or "ies", discard "es" and "ed" at the end.
    # if it has only 1 vowel or 1 set of consecutive vowels, discard. (like "speed", "fled" etc.)

    if word[-2:] == "es" or word[-2:] == "ed" :
        doubleAndtripple_1 = len(re.findall(r'[eaoui][eaoui]',word))
        if doubleAndtripple_1 > 1 or len(re.findall(r'[eaoui][^eaoui]',word)) > 1 :
            if word[-3:] == "ted" or word[-3:] == "tes" or word[-3:] == "ses" or word[-3:] == "ied" or word[-3:] == "ies" :
                pass
            else :
                disc+=1

    #3) discard trailing "e", except where ending is "le"

    le_except = ['whole','mobile','pole','male','female','hale','pale','tale','sale','aisle','whale','while']

    if word[-1:] == "e" :
        if word[-2:] == "le" and word not in le_except :
            pass

        else :
            disc+=1

    #4) check if consecutive vowels exists, triplets or pairs, count them as one.

    doubleAndtripple = len(re.findall(r'[eaoui][eaoui]',word))
    tripple = len(re.findall(r'[eaoui][eaoui][eaoui]',word))
    disc+=doubleAndtripple + tripple

    #5) count remaining vowels in word.
    numVowels = len(re.findall(r'[eaoui]',word))

    #6) add one if starts with "mc"
    if word[:2] == "mc" :
        syls+=1

    #7) add one if ends with "y" but is not surrouned by vowel
    if word[-1:] == "y" and word[-2] not in "aeoui" :
        syls +=1

    #8) add one if "y" is surrounded by non-vowels and is not in the last word.

    for i,j in enumerate(word) :
        if j == "y" :
            if (i != 0) and (i != len(word)-1) :
                if word[i-1] not in "aeoui" and word[i+1] not in "aeoui" :
                    syls+=1

    #9) if starts with "tri-" or "bi-" and is followed by a vowel, add one.

    if word[:3] == "tri" and word[3] in "aeoui" :
        syls+=1

    if word[:2] == "bi" and word[2] in "aeoui" :
        syls+=1

    #10) if ends with "-ian", should be counted as two syllables, except for "-tian" and "-cian"

    if word[-3:] == "ian" :
    #and (word[-4:] != "cian" or word[-4:] != "tian") :
        if word[-4:] == "cian" or word[-4:] == "tian" :
            pass
        else :
            syls+=1

    #11) if starts with "co-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:2] == "co" and word[2] in 'eaoui' :

        if word[:4] in co_two or word[:5] in co_two or word[:6] in co_two :
            syls+=1
        elif word[:4] in co_one or word[:5] in co_one or word[:6] in co_one :
            pass
        else :
            syls+=1

    #12) if starts with "pre-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:3] == "pre" and word[3] in 'eaoui' :
        if word[:6] in pre_one :
            pass
        else :
            syls+=1

    #13) check for "-n't" and cross match with dictionary to add syllable.

    negative = ["doesn't", "isn't", "shouldn't", "couldn't","wouldn't"]

    if word[-3:] == "n't" :
        if word in negative :
            syls+=1
        else :
            pass

    #14) Handling the exceptional words.

    if word in exception_del :
        disc+=1

    if word in exception_add :
        syls+=1

    # calculate the output
    return numVowels - disc + syls

In [32]:
##### Defining a function to calculate percentage of complex words and saving
##### it in the dataframe
def percent_complex(words):

  complex_words = [word for word in words if sylco(word) > 2]

  total_words = len(words)
  total_complex_words = len(complex_words)

  percentage_complex_words = (total_complex_words / total_words)*100 if total_words > 0 else 0
  return percentage_complex_words

In [33]:
output_file['PERCENTAGE OF COMPLEX WORDS']= output_file.apply(lambda row: percent_complex(row['Processed']), axis=1)

In [34]:
##### Defining a function to calculate fog index and saving it in dataframe
def fog(avg_sent_len, percentage_complex):
  fog_index = 0.4*(avg_sent_len + percentage_complex)
  return fog_index

In [35]:
output_file['FOG INDEX']= output_file.apply(lambda row: fog(row['AVG SENTENCE LENGTH'], row['PERCENTAGE OF COMPLEX WORDS']), axis=1)

In [36]:
##### Saving the average number of words per sentence in dataframe
output_file['AVG NUMBER OF WORDS PER SENTENCE']=output_file['AVG SENTENCE LENGTH']

In [37]:
##### Defining a function to calculate number of complex words and saving it in
##### dataframe
def no_complex(words):
  complex_words = [word for word in words if sylco(word) > 2]
  total_complex_words = len(complex_words)

  return total_complex_words

In [38]:
output_file['COMPLEX WORD COUNT']= output_file.apply(lambda row: no_complex(row['Processed']), axis=1)

In [41]:
##### Defining a function to clean the text by removing puntuation, tokenizing
##### it and removing stopwords defined in the nltk library. The function
##### returns the number of clean words after performing the above operations
def text_clean(content):
  corp = re.sub('[^a-zA-Z]+', ' ', content).strip()
  tokens = word_tokenize(corp)
  words = [t for t in tokens if t not in stopwords]
  return len(words)

In [42]:
##### Saving the clean words count using the above function
output_file['WORD COUNT']= output_file.apply(lambda row: text_clean(row['Data']), axis=1)

In [43]:
##### Defining a function to calculate the average number of syllables per word
##### and saving it in dataframe
def syllable_per_word(words):
  no_of_syllables=0
  for word in words:
    no_of_syllables=no_of_syllables+sylco(word)
  avg_syllables=no_of_syllables/len(words) if len(words)>0 else 0
  return avg_syllables

In [44]:
output_file['SYLLABLE PER WORD']= output_file.apply(lambda row: syllable_per_word(row['Processed']), axis=1)

In [45]:
##### Defining a function to calculate the number of personal pronouns and
##### saving it in dataframe
def cal_personal_pronouns(content):
  pronoun_re = r'\b(I|my|My|we|We|us|Us|ours|Ours)\b'
  matches = re.findall(pronoun_re, content)
  return len(matches)

In [46]:
output_file['PERSONAL PRONOUNS']= output_file.apply(lambda row: cal_personal_pronouns(row['Data']), axis=1)

In [47]:
##### Defining a function to calculate the average word length and saving it in
##### dataframe
def avg_word_length(words):
  no_of_characters=0
  for word in words:
    no_of_characters=no_of_characters+len(word)
  avg_word_len=no_of_characters/len(words)
  return avg_word_len

In [48]:
output_file['AVG WORD LENGTH']= output_file.apply(lambda row: avg_word_length(row['Processed']), axis=1)

In [49]:
##### Printing the first five rows of dataframe
output_file.head()

,URL_ID,URL,Data,Processed,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,Rising IT cities and its impact on the economy...,"[rising, cities, impact, economy, environment,...",27,6,0.636364,0.067901,6.151899,31.893004,15.217961,6.151899,155,608,2.257202,11,6.915638
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,Rising IT Cities and Their Impact on the Econo...,"[rising, cities, impact, economy, environment,...",52,31,0.253012,0.121701,8.525000,44.428152,21.181261,8.525000,303,772,2.565982,4,7.768328
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"Internet Demand's Evolution, Communication Imp...","[internet, demand, evolution, communication, i...",36,25,0.180328,0.099349,10.771930,51.465798,24.895091,10.771930,316,666,2.741042,13,8.276873
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,Rise of Cybercrime and its Effect in upcoming ...,"[rise, cybercrime, effect, upcoming, future, b...",35,74,-0.357798,0.185374,11.307692,50.680272,24.795186,11.307692,298,623,2.693878,5,8.204082
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,OTT platform and its impact on the entertainme...,"[platform, impact, entertainment, industry, fu...",23,9,0.437500,0.093023,8.600000,36.337209,17.974884,8.600000,125,395,2.360465,6,7.764535


In [50]:
##### Dropping the columns containing web scraped data anf processed data and
##### saving the dataframe as a csv file
output_file.drop(['Data', 'Processed'], axis=1, inplace=True)

In [51]:
output_file.to_csv('output_file.csv', index=False)